In [ ]:
# http://vision.stanford.edu/aditya86/ImageNetDogs/images.tar

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import os
import PIL
import tensorflow as tf

from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential


In [ ]:
!wget --no-check-certificate http://vision.stanford.edu/aditya86/ImageNetDogs/images.tar

In [ ]:
import tarfile 
f = tarfile.open("images.tar")
f.extractall('.')
f.close()

In [ ]:
requires_class = [
                  "n02085782-Japanese_spaniel",
                  "n02085620-Chihuahua",
                  "n02087046-toy_terrier"
]

In [ ]:
import os
import shutil

if not os.path.exists('datsets'):
  os.mkdir('datsets')

for dir_name in requires_class:
  src = f"Images/{dir_name}"
  dst = f"datasets/{dir_name}"

  shutil.copytree(src, dst)

In [ ]:
base_dir = "datasets/"

In [ ]:
import pathlib

data_directory = pathlib.Path(base_dir)
total_imgs = len(list(data_directory.glob('*/*.*')))
print(f"Total images: {total_imgs}")

**batch_size** = 32
img_width = 180
img_geight = 180

In [ ]:
batch_size = 32
img_width = 180
img_beight = 180

In [ ]:
training_dataset = tf.keras.preprocessing.image_dataset_from_directory(
    data_directory,
    class_names = list(sorted(requires_class)),
    image_size = (img_width, img_beight),
    validation_split = 0.2,
    subset="training",
    seed=123
)

In [ ]:
class_name = training_dataset.class_names
total_classes = len(class_name)

print(f"class names are:{class_name}")
print(f"total class are: {total_classes}")

In [ ]:
data_augmentation = keras.Sequential(
    [
     layers.experimental.preprocessing.RandomFlip("horizontal", input_shape=(img_width, img_beight, 3)),
     layers.experimental.preprocessing.RandomRotation(0.1),
     layers.experimental.preprocessing.RandomZoom(0.1),    ])

In [ ]:
model =  Sequential([
                     data_augmentation,
                     layers.experimental.preprocessing.Rescaling(1.0/255.0),
                     layers.Conv2D(16, 3, padding="same", activation="relu"),
                     layers.MaxPool2D(),
                     layers.Conv2D(32, 3, padding="same", activation="relu"),
                     layers.MaxPool2D(),
                     layers.Conv2D(64, 3, padding="same", activation="relu"),
                     layers.MaxPool2D(),
                     layers.Dropout(0.2),
                     layers.Flatten(),
                     layers.Dense(128, activation="relu"),
                     layers.Dense(total_classes)                     ])

In [ ]:
model.compile(optimizer="adam",
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),              metrics=['accuracy'])

In [ ]:
epochs = 30

ep_history = model.fit(
    training_dataset,
    validation_data = training_dataset,
    epochs=epochs
 )

In [ ]:
import glob

import random

all_img = glob.glob(f"{base_dir}/**/*.jpg")

In [ ]:
random_image = random.choice(all_img)

print(f"Random image:{random_image}")

img = keras.preprocessing.image.load_img(
    random_image,
    target_size=(img_width, img_beight)
)

img_array = keras.preprocessing.image.img_to_array(img)
img_array = tf.expand_dims(img_array, 0)

prediction = model.predict(img_array)
score = tf.nn.softmax(prediction[0])

print(f"predictions : {prediction}")
print(f"Score: {score}")

prediction_name = class_name[np.argmax(score)]
per = np.max(score) * 100

print(f"image belongs to {prediction_name} with {per: .2f}% confidence")



In [ ]:
model.save('dog-breed.h5')